In [1]:
from sklearn.metrics import f1_score
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.cluster import KMeans

from torch.utils.data import TensorDataset, DataLoader

In [2]:
test_data = np.load('dataset/test-release.npy', allow_pickle=True).item()
train_split_data = np.load('dataset/train_split.npy', allow_pickle=True).item()
val_split_data = np.load('dataset/val_split.npy', allow_pickle=True).item()

In [3]:
test_data['sequences']['3c8a57ba51']['keypoints'].shape

(19492, 2, 2, 7)

In [5]:
test_data['sequences']['3c8a57ba51']['annotator_id']

0

In [6]:
# train data processing
train_features = []
train_labels = []

for sequence in train_split_data['sequences'].values():
    keypoints = sequence['keypoints']
    annotations = sequence['annotations']

    num_frames = keypoints.shape[0]
    keypoints[:, :, 0, :] /= 1024.0
    keypoints[:, :, 1, :] /= 570.0
    features_flat = keypoints.reshape(num_frames, -1)

    train_features.append(features_flat)
    train_labels.append(annotations)

train_features = np.concatenate(train_features, axis=0)
train_labels = np.concatenate(train_labels, axis=0)


print(train_features.shape)
print(train_labels.shape)

(426635, 28)
(426635,)


In [7]:
# validation data processing
val_features = []
val_labels = []

for sequence in val_split_data['sequences'].values():
    keypoints = sequence['keypoints']
    annotations = sequence['annotations']

    num_frames = keypoints.shape[0]
    keypoints[:, :, 0, :] /= 1024.0
    keypoints[:, :, 1, :] /= 570.0
    features_flat = keypoints.reshape(num_frames, -1)

    val_features.append(features_flat)
    val_labels.append(annotations)

val_features = np.concatenate(val_features, axis=0)
val_labels = np.concatenate(val_labels, axis=0)


print(val_features.shape)
print(val_labels.shape)

(81103, 28)
(81103,)


In [8]:
# test data processing
test_features = []

for sequence in test_data['sequences'].values():
    keypoints = sequence['keypoints']

    num_frames = keypoints.shape[0]
    keypoints[:, :, 0, :] /= 1024.0
    keypoints[:, :, 1, :] /= 570.0
    features_flat = keypoints.reshape(num_frames, -1)

    test_features.append(features_flat)

test_features = np.concatenate(test_features, axis=0)

print(test_features.shape)

(8168491, 28)


In [9]:
# combine train and val data
combined_features = np.concatenate([train_features, test_features], axis=0)
print(combined_features.shape)

(8595126, 28)


In [10]:
# cluster the combined features into 20 clusters
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(combined_features)

print(cluster_labels.shape)
np.save('./dataset/cluster_labels.npy', cluster_labels)

(8595126,)


In [11]:
class FC_classifier(nn.Module):
    def __init__(self, input_dim):
        super(FC_classifier, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc_final_20 = nn.Linear(32, 20)
        self.fc_final_4 = nn.Linear(32, 4)

    def forward(self, x, is_combined_features=False):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        if is_combined_features:
            x = self.fc_final_20(x)
        else:
            x = self.fc_final_4(x)
        return x

In [12]:
combined_labels = np.load('./dataset/cluster_labels.npy')

In [13]:
def validation(model, val_features, val_labels, device='cuda' if torch.cuda.is_available() else 'mps'):
    X = torch.tensor(val_features, dtype=torch.float32)
    y = torch.tensor(val_labels, dtype=torch.long)
    
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=256, shuffle=True)
    
    model = model.to(device)
    model.eval()
    
    total_predictions = 0
    correct_predictions = 0
    labels = []
    total_predicted = []
    with torch.no_grad():
        for batch_X, batch_y in dataloader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            batch_X = batch_X.view(batch_X.size(0), -1)
            outputs = model(batch_X)
            
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += batch_y.size(0)
            correct_predictions += (predicted == batch_y).sum().item()
            labels = labels + batch_y.tolist()
            total_predicted = total_predicted + predicted.tolist()
    F1 = f1_score(labels, total_predicted, average='macro', labels=[0, 1, 2])
    print(f"Val-F1: {F1:.4f}")
    return F1

In [14]:
def train_model(model, train_features, train_labels, combined_features, combined_labels, epochs=10, batch_size=256, lr=1e-3, device='cuda' if torch.cuda.is_available() else 'mps'):
    train_features = torch.tensor(train_features, dtype=torch.float32)
    train_labels = torch.tensor(train_labels, dtype=torch.long)
    combined_labels = torch.tensor(combined_labels, dtype=torch.long)
    combined_features = torch.tensor(combined_features, dtype=torch.float32)
    
    train_dataset = TensorDataset(train_features, train_labels)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    combined_dataset = TensorDataset(combined_features, combined_labels)
    combined_dataloader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=True)
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_predictions = 0
        correct_predictions = 0
        labels = []
        total_predicted = []
        for batch_X, batch_y in train_dataloader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            batch_X = batch_X.view(batch_X.size(0), -1)
            optimizer.zero_grad()
            outputs = model(batch_X, is_combined_features=False)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += batch_y.size(0)
            correct_predictions += (predicted == batch_y).sum().item()
            total_loss += loss.item() * batch_X.size(0)
            labels = labels + batch_y.tolist()
            total_predicted = total_predicted + predicted.tolist()
        for batch_X, batch_y in combined_dataloader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            batch_X = batch_X.view(batch_X.size(0), -1)
            optimizer.zero_grad()
            outputs = model(batch_X, is_combined_features=True)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
        avg_loss = total_loss / len(train_dataset)
        accuracy = 100 * correct_predictions / total_predictions
        F1 = f1_score(labels, total_predicted, average='macro', labels=[0, 1, 2])
        print(f"Epoch {epoch+1}/{epochs} - Train-Loss: {avg_loss:.4f} - Train-Accuracy: {accuracy:.2f}% - Train-F1: {F1:.4f}")
        validation(model, val_features, val_labels, device='cuda' if torch.cuda.is_available() else 'mps')
    print("Training complete.")
    return model

In [15]:
input_dim = 28
model = FC_classifier(input_dim=input_dim)
model = train_model(model, train_features, train_labels, combined_features, combined_labels, epochs=50, batch_size=256, lr=1e-3, device='cuda' if torch.cuda.is_available() else 'mps')

Epoch 1/50 - Train-Loss: 0.4932 - Train-Accuracy: 81.25% - Train-F1: 0.4487
Val-F1: 0.0438
Epoch 2/50 - Train-Loss: 0.5939 - Train-Accuracy: 82.13% - Train-F1: 0.4328
Val-F1: 0.1108
Epoch 3/50 - Train-Loss: 0.4606 - Train-Accuracy: 84.36% - Train-F1: 0.4946
Val-F1: 0.0958
Epoch 4/50 - Train-Loss: 0.4410 - Train-Accuracy: 84.68% - Train-F1: 0.4999
Val-F1: 0.1491
Epoch 5/50 - Train-Loss: 0.4293 - Train-Accuracy: 85.12% - Train-F1: 0.5070
Val-F1: 0.1616
Epoch 6/50 - Train-Loss: 0.4236 - Train-Accuracy: 85.26% - Train-F1: 0.5092
Val-F1: 0.1696
Epoch 7/50 - Train-Loss: 0.4223 - Train-Accuracy: 85.16% - Train-F1: 0.5100
Val-F1: 0.1817
Epoch 8/50 - Train-Loss: 0.4192 - Train-Accuracy: 85.13% - Train-F1: 0.5077
Val-F1: 0.1747
Epoch 9/50 - Train-Loss: 0.4049 - Train-Accuracy: 85.57% - Train-F1: 0.5144
Val-F1: 0.2195
Epoch 10/50 - Train-Loss: 0.4012 - Train-Accuracy: 85.70% - Train-F1: 0.5182
Val-F1: 0.1978
Epoch 11/50 - Train-Loss: 0.4014 - Train-Accuracy: 85.66% - Train-F1: 0.5147
Val-F1: 0.21

KeyboardInterrupt: 